In [1]:
from gym.wrappers import FlattenObservation
import torch
from torch import nn
from torch import optim

from environments.absolute import ARESEAAbsolute

initializing ocelot...


In [2]:
env = ARESEAAbsolute()
env = FlattenObservation(env)

In [3]:
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

policy = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, act_dim)
        )
optimizer = optim.Adam(policy.parameters())

In [4]:
def rollout(env, policy, batch_size):
    observations = torch.zeros(batch_size, env.observation_space.shape[0])
    actions = torch.zeros(batch_size, env.action_space.shape[0])
    objectives = torch.zeros(batch_size)

    for i in range(batch_size):
        observation = env.reset()
        observation = torch.tensor(observation, dtype=torch.float32)
        observations[i] = observation

        observation = torch.unsqueeze(observation, 0)
        action = policy(observation)
        action = torch.squeeze(action)
        actions[i] = action
        
        action = action.detach().numpy()
        _, objective, _, _ = env.step(action)
        objectives[i] = objective
    
    return observations, actions, objectives

In [5]:
n_steps = 1000
batch_size = 64

for i in range(n_steps):
    observations, actions, objectives = rollout(env, policy, batch_size)

    loss = objectives.mean()
    
    policy.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Finished step {i}")

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [ ]:
class GaussianActor(nn.Module):

    def __init__(self, obs_dim, act_dim):
        super().__init__()

        self.log_std = nn.Parameter(0.5 * torch.ones(act_dim))
        self.mu_net = nn.Sequential(
            nn.Linear(obs_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, act_dim)
        )
    
    def forward(self, observation, action=None):
        mu = self.mu_net(observation)
        std = torch.exp(self.log_std)
        pi = distributions.Normal(mu, std)

        if action is None:
            return pi
        else:
            log_probs = pi.log_prob(action).sum(axis=-1)
            return pi, log_probs


class OneShotPolicyGradient:

    def __init__(self, env, batch_size=64):
        self.env = env
        self.batch_size = batch_size

        self.actor = GaussianActor(env.observation_space.shape, env.action_space.shape)
        self.optimizer = optim.Adam(self.actor.parameters)
    
    def learn(self, n_steps=1000):
        step = 0
        while step < n_steps:
            
            
            

            step += self.batch_size


if __name__ == "__main__":
    env = ARESEAAbsolute()
    env = FlattenObservation(env)